# Factor Analysis Framework

<img src="whiteboard.jpeg">

We will assume that for each individual, we observe $m$ different test $y$ and that there are $p$ anobserved realizations $z$. In total, we have $n$ different individuals. This variables are related as follows:

- $ Y|Z \sim N\left(\sideset{^{CZ}}{}\mu + \sideset{^{CZ}}{}{L}, \sideset{^{CZ}}{}\Sigma \right) $  
- $Z \sim N\left(0, 1 \right)$ 

where:
- $Cov(z_i, z_j) =\delta_{i,j}$. 

We will also assume that $\sideset{^{CZ}}{}\Sigma$ is a diagonal matrix.

The goal is, given $n$ observations of $Y$ $i.i.d$, calculate  $\Theta = \left \{\sideset{^{CZ}}{}\mu, \sideset{^{CZ}}{}L, \sideset{^{CZ}}{}\Sigma \right \}$  such that:

$$ \Theta = argmax \left \{ P \left(Y_1, ... , Y_n | \Theta \right) \right \} $$

The calculation strategy is as follows:

- For a given candidate $\Theta = \left \{\sideset{^{CZ}}{}\mu, \sideset{^{CZ}}{}L, \sideset{^{CZ}}{}\Sigma \right \}$ we calculate the joined distribution of $(Z, Y)\sim N\left \{\sideset{^{J}}{}\mu, \sideset{^{J}}{}\Sigma \right \}$

- Next, we must calculate the posterior probability distribution of $Z|Y$ i.e. $Z \sim N \left(\sideset{^{CY}}{}\mu, \sideset{^{CY}}{}\Sigma \right )$

- At this point we will be able to apply the EM algorithm. i.e. calculate:

$$ Q \left( \Theta_{new}, \Theta_{old} \right) = E_{Z|Y,\Theta_{old}} \left[ \log P\left( Z, Y | \Theta_{new} \right)  \right]$$

The reader will have noticed that we are using a top left script notation to specify at which point of the calculation we are, __CZ__ for conditioned on Z, __J__ for join distribution and __CY__ for conditioned on Y.

The following sections derived the calculation of $\sideset{^{J}}{}P$ and $\sideset{^{CY}}{}P$. We then present the implementation of the calculation framework and finally we show the result of randomly generated data.

## Calculation of the join distribution

Since Y|Z is a multivariated normal distribution, Z is also a multivariated normal distribution and the relation between Y and Z is linear, the join distribution is also in multivariate normal distribution. So to calculate the join distribution, all we need to do is calculate $\sideset{^{J}}{} \mu$ and  $\sideset{^{J}}{} \Sigma$.

We will define:

$$ X = \begin{pmatrix}
Z \\ Y
\end{pmatrix}
$$ 

### Calculation of $\sideset{^{J}}{} \mu$
Since $Z \sim N\left(0, 1 \right)$ it's easy to see that:

$$ \sideset{^{J}}{} \mu = \begin{pmatrix}
0_p \\ 
\sideset{^{CZ}}{} \mu
\end{pmatrix} $$ 

Where $0_p$ is a vector of dimension p with all its elements equal to 0.

### Calculation of $\sideset{^{J}}{} \Sigma $

Given that $y_i = \mu_i + \sum_k l_{i, k}z_k + \sigma_i * \epsilon_i$ all we need to do is calculate $Cov(x_i, x_j)$.

- $ Cov(z_i, z_j) = \delta_{i, j} $
- $ Cov(y_i, z_j) = l_{i,j} $
- $ Cov(y_i, y_i) = \sum_k l_{i, k}^2 + \sigma_i^2$
- $ Cov(y_i, y_j) = \sum_k l_{i, k} l_{j, k} $

Where $\delta_{i,j}$ is the kronecker delta.

## Calculations of the probability distribution of Z conditioned on Y

Let  
$$\sideset{^{J}}{} \Sigma = \begin{pmatrix} 
                    \sideset{^{J}}{} \Sigma_{1, 1} & \sideset{^{J}}{} \Sigma_{1, 2} \\
                    \sideset{^{J}}{} \Sigma_{2, 1} & \sideset{^{J}}{} \Sigma_{2, 2}
                    \end{pmatrix} \text{ with sizes }
                    \begin{pmatrix} 
                    p \times p & p \times m \\
                    m \times p & m \times m
                    \end{pmatrix}
                    $$

### Calculation of $\sideset{^{CY}}{} \mu$

<center>
    $\sideset{^{CY}}{}\mu =\sideset{^{J}}{}\Sigma_{1, 2} 
\sideset{^{J}}{} \Sigma_{2, 2}^{-1} (y - \sideset{^{CZ}}{}\mu)$ 
</center>

Since $Z \sim N(0, 1)$

### Calculation of $\sideset{^{CY}}{} \Sigma$


                    

<center>
    $\sideset{^{CY}}{} \Sigma =\sideset{^{J}}{} \Sigma_{1, 1} -\sideset{^{J}}{}\Sigma_{1, 2} 
\sideset{^{J}}{} \Sigma_{2, 2}^{-1} \sideset{^{J}}{} \Sigma_{2, 1}$ 
</center>


## Calculation of $ Q \left( \Theta_{new}, \Theta_{old} \right) $ 

In orther to compute $ Q \left( \Theta_{new}, \Theta_{old} \right) $ i.e. $ = E_{Z|Y,\Theta_{old}} \left[ \log P\left( Z, Y | \Theta_{new} \right)  \right]$ we will right the former expresion as an integral over $Z$ specifying the density function.

That is: 
<center>

$ Q \left( \Theta_{new}, \Theta_{old} \right)
\propto 
\int_{Z} \left (
-\frac{1}{2}(x - \sideset{^{J}}{}\mu_{new})'
\sideset{^{J}}{} \Sigma_{new}^{-1}
(x - \sideset{^{J}}{}\mu_{new}) 
- \frac{1}{2} \log |\sideset{^{J}}{} \Sigma_{new}| 
\right) 
\frac{exp
\left (
-\frac{1}{2}(x - \sideset{^{CY}}{}\mu_{old})'
\sideset{^{CY}}{} \Sigma_{old}^{-1}
(x - \sideset{^{CY}}{}\mu_{old}) 
\right) 
}
{\sqrt{|\sideset{^{CY}}{} \Sigma_{old}|}} dz$
    </center>

But this integral is fairly simple, because what we are integrating over $Z$ is a quadratic form on $X = (Z, Y)$ minus a function, $-\frac{1}{2} \log |\sideset{^{J}}{} \Sigma_{new}|$. This function on $\Theta_{new}$ is a constant when integrated over $Z$.

Now, the quadratic form has the following cases, $z_i$ vs $z_i$, $z_i$ vs $z_j$, $y_i$ vs $z_j$ and $y_i$ vs $y_j$

### $z_i$ vs $z_i$:


$ \begin{align}
E_{Z|Y,\Theta_{old}}[^{J}\Sigma^{-1}_{new, i, i}*(z_i - ^{J}\mu_{new, i})^2] 
&= E_{Z|Y,\Theta_{old}}[^{J}\Sigma^{-1}_{new, i, i}*(z_i - {^{CY}\mu_{old, i}} + {^{CY}\mu_{old, i}} - ^{J}\mu_{new, i})^2] \\
&= E_{Z|Y,\Theta_{old}}[^{J}\Sigma^{-1}_{new, i, i}*(z_i - {^{CY}\mu_{old, i}} + {^{CY}\mu_{old, i}})^2] \\
&= ^{J}\Sigma^{-1}_{new, i, i}*E_{Z|Y,\Theta_{old}}[(z_i - {^{CY}\mu_{old, i}} + {^{CY}\mu_{old, i}})^2] \\
&= ^{J}\Sigma^{-1}_{new, i, i}*E_{Z|Y,\Theta_{old}}[(z_i - {^{CY}\mu_{old, i}})^2 + 2(z_i - {^{CY}\mu_{old, i}}) + {^{CY}\mu_{old, i}^2}] \\
&= ^{J}\Sigma^{-1}_{new, i, i}*E_{Z|Y,\Theta_{old}}[(z_i - {^{CY}\mu_{old, i}})^2 + ^{CY}\mu_{old, i}^2] \\
&= ^{J}\Sigma^{-1}_{new, i, i}*(^{CY}\Sigma_{old, i, i} + {^{CY}\mu_{old, i}^2}) 
\end{align}
$


Since $ \sideset{^{J}}{} \mu_{new, i} = 0$ 


###  $z_i$ vs $z_j$:

$ \begin{align}
E_{Z|Y,\Theta_{old}}[^{J}\Sigma^{-1}_{new, i, j}(z_i -  ^{J}\mu_{new, i})(z_j - ^{J}\mu_{new, j})] 
&=2 ^{J}\Sigma_{new, i, j}^{-1} E_{Z|Y,\Theta_{old}}[(z_i -  {^{J}\mu_{new, i}})(z_j - ^{J}\mu_{new, j})] \\
&=2 ^{J}\Sigma_{new, i, j}^{-1} E_{Z|Y,\Theta_{old}}[(z_i - {^{CY}\mu_{old, i}} +  {^{CY}\mu_{old, i}}-  ^{J}\mu_{new, i})
(z_j - {^{CY}\mu_{old, j}}+ {^{CY}\mu_{old, j}}- {^{J}\mu_{new, j}})] \\
&=2 ^{J}\Sigma_{new, i, j}^{-1} E_{Z|Y,\Theta_{old}}[(z_i - {^{CY}\mu_{old, i}})(z_j - {^{CY}\mu_{old, j}})+({^{CY}\mu_{old, i}}-  ^{J}\mu_{new, i})({^{CY}\mu_{old, j}}- {^{J}\mu_{new, j}})] \\ 
&=2 ^{J}\Sigma_{new, i, j}^{-1} (^{CY}\Sigma_{old, i, j} + ({^{J}\mu_{old, i}}-  ^{J}\mu_{new, i})({^{CY}\mu_{old, j}}- {^{J}\mu_{new, j}})) \\
&=2 ^{J}\Sigma_{new, i, j}^{-1} (^{CY}\Sigma_{old, i, j} + {^{CY}\mu_{old, i}}{^{CY}\mu_{old, j}}) 
\end{align}$

Again since  $\sideset{^{J}}{} \mu_{new, i} = 0$.

### $y_i$ vs $z_j$:

$\begin{align}
E_{Z|Y,\Theta_{old}}[{^{J}\Sigma^{-1}_{new, i, j}}(z_i - {^{J}\mu_{new, i}})(y_j - {^{J}\mu_{new, j}})] 
&=2{^{J}\Sigma^{-1}_{new, i, j}}E_{Z|Y,\Theta_{old}}[(z_i - {^{J}\mu_{new, i}})(y_j - {^{J}\mu_{new, j}})] \\ 
&=2{^{J}\Sigma^{-1}_{new, i, j}}(y_j - {^{J}\mu_{new, j}})E_{Z|Y,\Theta_{old}}[(z_i - {^{J}\mu_{new, i}})] \\ 
&=2{^{J}\Sigma^{-1}_{new, i, j}}(y_j - {^{J}\mu_{new, j}})^{CY}\mu_{old, i}
\end{align}
$

Again since  $\sideset{^{J}}{} \mu_{new, i} = 0$.

### $y_i$ vs $y_j$:

$E_{Z|Y,\Theta_{old}}[\sideset{^{J}}{}  \Sigma_{new, i, j}^{-1}(y_i - \sideset{^{J}}{} \mu_{new, i})(y_j - \sideset{^{J}}{} \mu_{new, j})] = 2\sideset{^{J}}{}  \Sigma_{new, i, j}^{-1}(y_i - \sideset{^{J}}{} \mu_{new, i})(y_j - \sideset{^{J}}{} \mu_{new, j})$


### $y_i$ vs $y_i$:

$E_{Z|Y,\Theta_{old}}[\sideset{^{J}}{}  \Sigma_{new, i, i}^{-1}(y_i - \sideset{^{J}}{} \mu_{new, i})(y_i - \sideset{^{J}}{} \mu_{new, i})] = \sideset{^{J}}{}  \Sigma_{new, i, i}^{-1}(y_i - \sideset{^{J}}{} \mu_{new, i})^2$


## Implementation

The implementatios has been divided in three sections:
- __Factor Analysis Classes:__ Here we implement two classes. 
    - __MultivariateDistribution:__ Encapsulating all the machinery for probability distributions calculations i.e. Condition on Z, Join and Condition on Y.
    - __FactorAnalysisModel:__ Encapsulation the calculations of $Q(\theta_{new}, \theta_{old})$
- __Data Generation Process:__ Given ${^{CZ}\mu}$, ${^{CZ}L}$ and ${^{CZ}\Sigma}$ generates n samples of $Y$
- __Optimization Helper Functions:__ 
    - __<code>break_params:</code>__ Function for translating the <code>np.array</code> that the scipy minimizer is able to deal with to three different parmeters ${^{CZ}\mu}$, ${^{CZ}L}$ and ${^{CZ}\Sigma}$ that our class is able to manage. 
    - __<code>get_f:</code>__ Function for encapsulation all the object oriented programming into a function that the scipy minimizer will deal with.


## Importing Packages

In [1]:
import numpy as np
from scipy.optimize import minimize

## Factor Analysis Classes

In [2]:
class MultivariateDistribution(object):
	def __init__(self, Y):
		self.Y = Y
		self._CZ_MU = None
		self._CZ_LF = None
		self._CZ_SIG = None
		self._J_SIG = None
		self._CY_SIG = None
		self._J_sig_inv = None
		self._J_sig_log_det = None

	def set_theta(self, MU, LF, SIG):
		self._CZ_MU = MU
		self._CZ_LF = LF
		self._CZ_SIG = SIG
		self._J_SIG = None
		self._CY_SIG = None
		self._J_sig_inv = None
		self._J_sig_log_det = None

	def get_X(self):
		n = self.Y.shape[1]
		p = self.get_p()

		z = np.zeros((p, n))

		return np.concatenate((z, self.Y))

	def get_J_mu(self):
		p = self.get_p()
		z = np.zeros((p, 1))
		return np.concatenate((z, self._CZ_MU), axis=0)

	def get_J_sig(self):
		# return np.array [(p + m) x (p + m)]
		p = self.get_p()
		m = self.get_m()

		if self._J_SIG is None:
			sig = np.zeros((p+m, p+m))

			for i in range(p+m):
				for j in range(p+m):
					if i < p:
						if j < p:
							if i == j:
								sig[i, i] = 1
							else:
								pass
						else:
							sig[i, j] = self._CZ_LF[j-p, i] # ??????
					else:
						if j < p:
							sig[i, j] = self._CZ_LF[i-p, j] # ?????
						else:
							if i == j:
								for k in range(p):
									sig[i, j] += self._CZ_LF[i-p, k]**2 # ?????
								sig[i, j] += self._CZ_SIG[i-p]**2
							else:
								for k in range(p):
									sig[i, j] += self._CZ_LF[i-p, k]*self._CZ_LF[j-p, k] # ?????

			self._J_SIG = sig

		return self._J_SIG

	def get_CY_mu(self, y):
		p = self.get_p()
		m = self.get_m()
		y = y.reshape((m, 1))
		CZ_MU = self._CZ_MU
		J_SIG = self.get_J_sig()
		sig_12 = J_SIG[:p, p:]
		sig_22 = J_SIG[p:, p:]
		sig_22_inv = np.linalg.inv(sig_22)

		return sig_12@sig_22_inv@(y - CZ_MU)

	def get_J_sig_inv(self):
		if self._J_sig_inv is None:
			self._J_sig_inv = np.linalg.inv(self.get_J_sig())

		return self._J_sig_inv

	def get_CY_sig(self):
		# TODO:
		# return np.array [p x p]
		p = self.get_p()
		m = self.get_m()

		if self._CY_SIG is None:

			J_sig = self.get_J_sig()
			sig11 = J_sig[:p, :p]
			sig12 = J_sig[:p, p:]
			sig21 = J_sig[p:, :p]
			sig22 = J_sig[p:, p:]

			sig = sig11 - sig12 @ np.linalg.inv(sig22) @ sig21
			self._CY_SIG = sig

		return self._CY_SIG

	def get_J_sig_log_det(self):
		if self._J_sig_log_det is None:
			self._J_sig_log_det = np.log(np.linalg.det(self.get_J_sig()))

		return self._J_sig_log_det

	def get_p(self):
		return self._CZ_LF.shape[1]

	def get_m(self):
		return self._CZ_LF.shape[0]

class FactorAnalysisModel(object):
	def __init__(self, mvd_old, mvd_new):
		self.mvd_old = mvd_old
		self.mvd_new = mvd_new

	def set_theta(self, MU, LF, SIG):
		self.mvd_new.set_theta(MU, LF, SIG)

	def q_x(self, x):
		# y [m x 1]
		# x [(p + m) x 1]

		result = 0
		p = self.mvd_old.get_p()

		CY_mu_old = self.mvd_old.get_CY_mu(x[p:])
		CY_sig_old = self.mvd_old.get_CY_sig()
		J_mu_new = self.mvd_new.get_J_mu()
		J_sig_new_inv = self.mvd_new.get_J_sig_inv()

		# -1/2 (x-J_mu_new)' * J_sig_new_inv * (x -J_mu_new) -1/2 log(det(J_sig_new))
		# -1/2 log(det(J_sig_new))
		result += self.mvd_new.get_J_sig_log_det()

		# -1/2 (x-J_mu_new)' * J_sig_new_inv * (x -J_mu_new)
		for i in range(J_sig_new_inv.shape[0]):
			for j in range(J_sig_new_inv.shape[1]):
				if (i < p) and (j < p):
					# We are in Z,Z
					result += J_sig_new_inv[i, i] * (CY_sig_old[i, j] + CY_mu_old[i] * CY_mu_old[j])
				elif (i >= p) and (j >= p):
					result += J_sig_new_inv[i, j] * (x[i] - J_mu_new[i])*(x[j] - J_mu_new[j])
				else:
					if i < j:
						result += J_sig_new_inv[i, j] * (x[j] - J_mu_new[j]) * CY_mu_old[i]
					else:
						result += J_sig_new_inv[i, j] * (x[i] - J_mu_new[i]) * CY_mu_old[j]

		return -1/2*result

	def q(self):
		result = 0
		X = self.mvd_old.get_X()
		for i in range(X.shape[1]):
			result += self.q_x(X[:, i])

		return result


## Data Generation Process

In [3]:
# Module is entitle to the generation of data given
# MU
# LOADING FACTORS
# SIGMA

def gen_data(MU, LF, SIG, n):
	# TODO
	# Y ~ N(MU + LF*Z, SIG)
	# MU
	# LOADING FACTORS
	# SIGMA
	# n: number of generated Ys
	# return a [m x n] matrix
	m = MU.shape[0]
	p = MU.shape[1]

	epsilons = np.random.normal(0, 1, (m, n))
	epsilons = epsilons*SIG.reshape((m, 1))

	Zs = np.random.normal(0, 1, (p, n))

	Ys = MU + LF@Zs + epsilons

	return np.array(Ys)

## Optimization Helper Functions

In [4]:
def break_params(theta, y_dim, z_dim):
	# TODO
	# MU <- theta()
	# LF <- theta()
	# SIG <- theta()

	MU = theta[:y_dim].reshape((y_dim, 1))
	LF = theta[y_dim:-y_dim].reshape((y_dim, z_dim))
	SIG = theta[-y_dim:]**2 # + 0.00000001 # This **2 is to avoid restrictions on the optimizer
	return MU, LF, SIG


def get_f(fam, y_dim, z_dim):

	def f(theta):
		MU, LF, SIG = break_params(theta, y_dim, z_dim)
		fam.mvd_new.set_theta(MU, LF, SIG)
		return -fam.q()

	return f



## Running Example


We will generate the following data set, dim(y)=2, dim(Z)=1

    CZ_MU = np.zeros((2, 1))
	CZ_L = np.matrix([[1], [3]])
	CZ_SIG = np.array([1, 1])
    
Our initial guess is 

    CZ_MU = np.array([0.5, 0.5])
	CZ_L = np.matrix([[1.5], [3.5]])
	CZ_SIG = np.array([1.5, 1.5])


The number of samples is 1000

In [5]:
	y_dim = 2
	z_dim = 1
	CZ_MU = np.zeros((2, 1))
	CZ_L = np.matrix([[1], [3]])
	CZ_SIG = np.array([1, 1])
    
	n = 1000

	Y = gen_data(CZ_MU, CZ_L, CZ_SIG, n)

	mvd_old = MultivariateDistribution(Y)
	mvd_new = MultivariateDistribution(Y)

	init_guess = np.array([0.5, 0.5, 1.5, 3.5, 1.5, 1.5])
	mu_init, l_init, sig_init = break_params(init_guess, y_dim, z_dim)

	mvd_old.set_theta(mu_init, l_init, sig_init)
	mvd_new.set_theta(mu_init, l_init, sig_init)

	fam = FactorAnalysisModel(mvd_old, mvd_new)

	f = get_f(fam, y_dim, z_dim)

	res = minimize(f,
	               init_guess,
	               method='Nelder-Mead',
	               options={'maxiter': 100})


	for s in range(25):
		print(np.round(res.x, 4))
		mu, lf, sig = break_params(res.x, y_dim, z_dim)
		fam.mvd_old.set_theta(mu, lf, sig)
		#f = get_f(fam, y_dim, z_dim)
		res = minimize(f,
		               res.x,
		               method='Nelder-Mead',
		               options={'maxiter': 100})

	mu, lf, sig = break_params(res.x, y_dim, z_dim)

	print('Hey ho lets go')

[0.2941 0.5989 0.9918 3.355  1.1061 1.4102]
[0.1531 0.5045 1.0644 3.0236 0.9985 1.3334]
[0.1618 0.3595 1.0774 2.8879 0.9587 1.2957]
[0.1227 0.3406 1.0836 2.8516 0.9428 1.2783]
[0.1104 0.2617 1.0765 2.8224 0.9353 1.2711]
[0.092  0.211  1.0715 2.8078 0.932  1.268 ]
[0.077  0.1698 1.0688 2.7985 0.9308 1.2667]
[0.0641 0.1373 1.0668 2.792  0.9306 1.2661]
[0.0538 0.1104 1.0651 2.7881 0.9305 1.2658]
[0.0454 0.0884 1.064  2.7854 0.9305 1.2657]
[0.0386 0.0704 1.0633 2.7834 0.9306 1.2657]
[0.033  0.0558 1.0626 2.7823 0.9306 1.2657]
[0.0284 0.0437 1.0623 2.7815 0.9307 1.2657]
[0.0246 0.0337 1.0619 2.7808 0.9307 1.2658]
[0.0214 0.0258 1.0618 2.7802 0.9307 1.2657]
[0.019  0.0191 1.0615 2.7799 0.9307 1.2657]
[0.0169 0.0136 1.0615 2.7795 0.9308 1.2657]
[0.0153 0.0092 1.0614 2.7796 0.9307 1.2657]
[0.0138 0.0056 1.0614 2.7795 0.9307 1.2657]
[1.2500e-02 2.7000e-03 1.0613e+00 2.7792e+00 9.3070e-01 1.2657e+00]
[1.1400e-02 2.0000e-03 1.0615e+00 2.7793e+00 9.3070e-01 1.2656e+00]
[ 1.1300e-02 -1.2000e-03  1.

We can see that values are recovered.

At this point, with the parameters ${^{CZ}\mu} + {^{CZ}{L}}, {^{CZ}\Sigma}$ we would be able to assign to each individual an instance of the latent variables by calculating the maximum likelyhood estimator.